<a href="https://colab.research.google.com/github/reallygooday/60daysofudacity/blob/master/Securing_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Federated Learning with a Trusted Aggregator

https://colab.research.google.com/drive/1FclC0VFbA3kIOKQog2Hkv62U_Q538pxQ#scrollTo=Z6OW1doarwy8

In [2]:
!pip3 install syft

     |████████████████████████████████| 225kB 5.1MB/s 
     |████████████████████████████████| 204kB 37.9MB/s 
     |████████████████████████████████| 2.1MB 41.6MB/s 
     |████████████████████████████████| 81kB 24.6MB/s 
     |████████████████████████████████| 256kB 45.4MB/s 
     |████████████████████████████████| 389kB 35.0MB/s 
     |████████████████████████████████| 460kB 45.3MB/s 
     |████████████████████████████████| 266kB 41.5MB/s 
     |████████████████████████████████| 51kB 19.0MB/s 
     |████████████████████████████████| 122kB 44.4MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067063 sha256=75ae6a3eeb8593396a996294c4a3dc13606e25f6a07423cd3d167d7e2113b7c5
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=94affb856cd8f9da35352fad9e8eadb6ff3c2a6ce0593e5b0b8ae9552b543477
  Stored in di

In [3]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

W0805 11:49:34.445146 140096615036800 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0805 11:49:34.465718 140096615036800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0805 11:49:39.092980 140096615036800 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0805 11:49:39.094405 140096615036800 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0805 11:49:39.095901 140096615036800 base.py:628] Worker bob already exists. Replacing old worker which could 

In [0]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

In [0]:
for i in range(10):

    # Train Bob's Model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()

    bobs_opt.step()
    bobs_loss = bobs_loss.get().data

    # Train Alice's Model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()

    alices_opt.step()
    alices_loss = alices_loss.get().data
    alices_loss

In [0]:

alices_model.move(secure_worker)
bobs_model.move(secure_worker)

In [0]:
with th.no_grad():

    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())

In [10]:

iterations = 10
worker_iters = 5

for a_iter in range(iterations):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for wi in range(worker_iters):
        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    with th.no_grad():

        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0376) Alice:tensor(0.0004)
Bob:tensor(0.0264) Alice:tensor(3.1765e-05)
Bob:tensor(0.0189) Alice:tensor(9.5062e-07)
Bob:tensor(0.0138) Alice:tensor(2.0013e-05)
Bob:tensor(0.0101) Alice:tensor(3.9295e-05)
Bob:tensor(0.0076) Alice:tensor(4.9503e-05)
Bob:tensor(0.0057) Alice:tensor(5.1582e-05)
Bob:tensor(0.0043) Alice:tensor(4.8505e-05)
Bob:tensor(0.0033) Alice:tensor(4.2853e-05)
Bob:tensor(0.0025) Alice:tensor(3.6359e-05)


In [0]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [12]:

print(preds)
print(target)
print(loss.data)

tensor([[0.1375],
        [0.1111],
        [0.8603],
        [0.8339]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0784)


# Intro to Additive Secret Sharing

In [0]:
x = 5

In [14]:

bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

In [0]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

In [15]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

In [16]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [17]:

(bob_x_share + alice_x_share) % Q

5

# Build Methods for Encrypt, Decrypt, and Add¶


In [0]:
x_share = (2,5,7)

In [0]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
    shares.append(Q - (sum(shares) % Q) + x)
    
    return tuple(shares)

def decrypt(shares):
    return sum(shares) % Q

In [20]:
shares = encrypt(3)
shares

(15751562968624096004463, 14295701605635982613046, 17433995113260400355940)

In [21]:
decrypt(shares)

3

In [0]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [23]:

x = encrypt(5)
y = encrypt(7)
z = add(x,y)
decrypt(z)

12

# Intro to Fixed Precision Encoding

In [0]:
# fixed precision encoding
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [30]:
encode(3.5)

35000

In [31]:
decode(35000)

3.5

In [0]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

# Secret Sharing + Fixed Precision in PySyft

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

Secret Sharing Using PySyft

In [0]:
x = x.share(bob, alice, secure_worker)

In [35]:
bob._objects

{32350693793: tensor([1130906658150434575, 3818997743768966171, 3931676216224208333,
         3383295988158366887, 1908418937599608490])}

In [0]:
y = x + x

In [37]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:72955199289 -> bob:45932490487]
	-> (Wrapper)>[PointerTensor | me:55790211317 -> alice:27499233919]
	-> (Wrapper)>[PointerTensor | me:72507609294 -> secure_worker:74704070097]
	*crypto provider: me*

In [38]:
y.get()

tensor([ 2,  4,  6,  8, 10])

Fixed Precision using PySyft

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [40]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [44]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [46]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

Shared Fixed Precision

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [52]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])